
![title](logos/cesi_exia.png)

---



### **Internship Project: Access Control Prototype using Facial Biometry**
#### **Internship Period: 08/12/2021 - 03/02/2022**


|Authors|Country|City|School|Campus|Position|Created|Updated|
|---|---|---|---|---|---|---|---|
|MD. HEMCH|Algeria|Oran|CESI ALGERIE|Oran|Student|21/12/2021|21/12/2021|
|NK. BENAMARA|Algeria|Oran|CESI ALGERIE|Oran|Internship Supervisor|21/12/2021|21/12/2021|

---

|Packages|Version|
|---|---|
|Numpy|1.19.5|
|Opencv|4.5.4.60|
|Tensorflow-GPU|1.15|
|Keras|2.2.4|
|VGG-Face Keras|0.6|
|h5py|2.10.0|





In [1]:
#import Libraries
import cv2 as cv
import numpy as np
import pandas as pd
import os 
from glob import glob
from tqdm import tqdm

from datetime import datetime



from tensorflow.keras.preprocessing.image import load_img,img_to_array
from keras_vggface.vggface import VGGFace
from keras_vggface import utils

In [2]:
MAIN_PATH=os.getcwd()
GALLERY_PATH=MAIN_PATH+'/gallery'
DATASET_PATH=MAIN_PATH+'/dataset_cropped'
MODELS=['vgg16', 'resnet50']

In [12]:
def load_vggface_model(model_type):
    '''
    this function load a pretrained VGG Face model (VGG16 or RESNET50)
    '''
    return VGGFace(model=model_type, include_top=False, input_shape=(224, 224, 3), pooling='avg')

def extract_n_save(model_type, dataset_path, output_path, output_type):
    ''''
    this function extracts and saves features in a npy file
    '''
    print('----------------------------------------------------')
    print('The Feature Extraction Work Has Started Using The {} Prerained Mode'.format(model_type.upper()))
    print('----------------------------------------------------')
    if model_type=='vgg16':
        preprocess_vgg_version=1
    elif model_type=='resnet50':
        preprocess_vgg_version=2

    supported_output_extensions=['npy', 'csv']
    vgg_face_model=load_vggface_model(model_type)
    feature_vectors=[]
    labels=[]
    directories=os.listdir(dataset_path)
    
    for directory in directories:
        print('Extracting features from {}'.format(str(directory)))
        os.chdir(dataset_path+'/'+directory)
        images=glob('*')
        for image in tqdm(images):
            img=load_img(image,target_size=(224,224)) #load an image and resize it to 224,224 like vgg face input size
            img=img_to_array(img) #convert the image to an array
            img=np.expand_dims(img,axis=0) #add the 4th dimension as a tensor to inject through the vgg face network
            img=utils.preprocess_input(img, version=preprocess_vgg_version) #preprocess the image
            feature_vector=vgg_face_model.predict(img) #extract the features

            feature_vectors.append(feature_vector) #append the current feature vector to the gallery
            labels.append(str(directory)) #append the current label to the gallery
    
    feature_vectors=np.squeeze(np.array(feature_vectors), axis=1)
    labels=np.expand_dims(np.array(labels),axis=1)


    if output_type not in supported_output_extensions:
        print('The output extenstion is incorrect. The available extensions are : {}'.format(supported_output_extensions))
    else:
        data_filename= model_type+'_x-train_'+datetime.now().strftime('%Y_%m_%d-at-%H_%M_%S.'+ output_type)
        labels_filename= model_type+'_y-train_'+datetime.now().strftime('%Y_%m_%d-at-%H_%M_%S.'+ output_type)

        print('--------------------------------')
        print('Feature Extraction Work is Finished Using the {} Pretrained Mode'.format(model_type.upper()))
        print('--------------------------------')
        print('Number of extracted features: {}'.format(np.shape(feature_vectors)[0]))
        print('Number of classes: {}'.format(np.shape(directories)[0]))

        if output_type=='npy':
    
            np.save(output_path+'/'+data_filename,feature_vectors)
            np.save(output_path+'/'+labels_filename,labels)

            print('Saved to : {}/{}'.format(output_path,data_filename))
            print('Saved to : {}/{}'.format(output_path,labels_filename))
    
        elif output_type=='csv':
            csv_data=np.concatenate((feature_vectors, labels), axis=1)
            pd.DataFrame(csv_data).to_csv(output_path+'/'+data_filename)
            
            print('Saved to : {}/{}'.format(output_path,data_filename))





In [13]:
for MODEL in MODELS:
    extract_n_save(MODEL,DATASET_PATH,GALLERY_PATH,'npy')

----------------------------------------------------
The Feature Extraction Work Has Started Using The VGG16 Prerained Mode
----------------------------------------------------
npy
False
Extracting features from Bale


100%|██████████| 49/49 [00:02<00:00, 21.81it/s]


Extracting features from Denzel


100%|██████████| 49/49 [00:01<00:00, 37.64it/s]


Extracting features from Gosling


...

Extracting features from Pitt


100%|██████████| 50/50 [00:01<00:00, 28.86it/s]


Extracting features from Statham


100%|██████████| 48/48 [00:01<00:00, 39.37it/s]


TypeError: unsupported operand type(s) for |: 'str' and 'str'